# Questionnaire processing

The purpose of this code is to retrieve the questionnaire data and export it to a cleaned CSV. We will use this CSV file in our scripts to compare the questoinnaire results.

## Load necessary packages

In [1]:
import pandas as pd
import numpy as np
import glob
import statsmodels

## Read in .csv data

In [2]:
sample = pd.read_csv('../Data/quest_raw/Participant_5_TSN_PostQ_Feb_13_2020_2021_Dec_08_1302.csv',
           usecols=['itemIndex', 'questions', 'ratings'],
           skiprows=[1,26])

In [3]:
questions = sample.questions

In [4]:
df = pd.DataFrame(columns=sample['itemIndex'])

for f in glob.glob('../Data/quest_raw/Part*.csv'):
    df = df.append([pd.read_csv(f, usecols = ['questions','ratings'], skiprows = [1,26])['ratings']], ignore_index=True)

In [5]:
df = df.drop(labels=[24], axis=1)

## Data Cleaning
### Replace columns

In [6]:
dat = df.rename(columns={
    0: 'gender',
    1: 'skill',
    2: 'hand',
    3: 'english_first',
    4: 'risk_appetite_1',
    5: 'risk_appetite_2',
    6: 'threat_susceptibility_1',
    7: 'risk_appetite_3',
    8: 'risk_appetite_4',
    9: 'malware_risk_1',
    10:'threat_severity_1',
    11:'malware_risk_2',
    12:'threat_severity_2',
    13:'malware_risk_3',
    14:'threat_susceptibility_2',
    15:'threat_severity_3',
    16:'risk_appetite_5',
    17:'risk_appetite_6',
    18:'threat_susceptibility_3',
    19:'have_antivirus',
    20:'updated_antivirus',
    21:'updated_os',
    22:'check_computer',
    23:'have_firewall',
    
})

### Add Participant Numbers

In [7]:
dat['subject'] = ['Participant_4',
                  'Participant_5',
                  'Participant_6',
                  'Participant_7',
                  'Participant_8',
                  'Participant_9',
                  'Participant_10',
                  'Participant_11',
                  'Participant_12',
                  'Participant_13',
                  'Participant_14',
                  'Participant_15',
                  'Participant_16',
                  'Participant_17',
                  'Participant_18',
                  'Participant_19',
                  'Participant_20',
                  'Participant_21',
                  'Participant_22',
                  'Participant_23',
                  'Participant_24'
                 ]

### Change to variables

In [8]:
dat['skill'] = dat['skill'].replace(['not at all', 
                                   'almost not at all', 
                                   'very little', 
                                   'neutral', 
                                   'a bit',
                                   'very much',
                                   'extremely'], [1,2,3,4,5,6,7])

dat['risk_appetite_2'] = dat['risk_appetite_2'].replace(['very easy',
                                                         'easy',
                                                         'neither easy or difficult',
                                                         'difficult',
                                                         'very difficult'], [1,2,3,4,5])


dat = dat.replace(['strongly disagree', 'disagree', 'neutral', 'agree', 'strongly agree'], [1,2,3,4,5])

dat['risk_appetite_1'] = dat['risk_appetite_1'].astype(int)

### Add ages
These are manually assigned because it is very painful to convert from PsychoPy's format for recording number key inputs. Everything else is much easier. These values can be retrieved manually from each of the PsychoPy questionnaire files.

In [9]:
dat['age'] = [26,25,25,25,23,20,26,20,21,18,20,20,20,20,21,18,28,21,28,21,20]

### Sort columns alphabetically

In [10]:
dat = dat.sort_index(axis=1, ascending=True)
dat.head()

,age,check_computer,english_first,gender,hand,have_antivirus,have_firewall,malware_risk_1,malware_risk_2,malware_risk_3,...,skill,subject,threat_severity_1,threat_severity_2,threat_severity_3,threat_susceptibility_1,threat_susceptibility_2,threat_susceptibility_3,updated_antivirus,updated_os
0,26,yes,yes,female,right,yes,yes,5,5,5,...,6,Participant_4,5,5,5,5,5,5,yes,yes
1,25,yes,no,male,right,yes,yes,4,4,4,...,6,Participant_5,5,5,5,3,3,3,yes,yes
2,25,no,yes,female,right,no,no,4,2,2,...,5,Participant_6,3,3,2,3,3,2,no,no
3,25,no,no,female,right,no,no,5,5,5,...,5,Participant_7,4,4,4,2,4,2,no,no
4,23,no,yes,female,right,no,no,4,3,4,...,4,Participant_8,4,4,4,3,4,2,no,no


### Descriptive statistics

In [11]:
dat.describe()

,age,malware_risk_1,malware_risk_2,malware_risk_3,risk_appetite_1,risk_appetite_2,risk_appetite_3,risk_appetite_4,risk_appetite_5,risk_appetite_6,skill,threat_severity_1,threat_severity_2,threat_severity_3,threat_susceptibility_1,threat_susceptibility_2,threat_susceptibility_3
count,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000
mean,22.190476,3.952381,3.523810,3.857143,4.476190,3.190476,3.952381,2.952381,3.238095,2.380952,4.952381,3.761905,3.619048,3.238095,2.666667,3.523810,2.619048
std,3.124405,0.589592,0.980767,0.654654,1.209093,0.749603,1.071270,0.864650,0.995227,0.864650,1.203170,0.830949,0.804748,0.943650,0.912871,0.980767,0.920662
min,18.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1.000000,2.000000,2.000000,1.000000,1.000000,2.000000,2.000000,2.000000,1.000000,1.000000,2.000000
25%,20.000000,4.000000,3.000000,4.000000,4.000000,3.000000,4.000000,2.000000,2.000000,2.000000,5.000000,3.000000,3.000000,3.000000,2.000000,3.000000,2.000000
50%,21.000000,4.000000,4.000000,4.000000,5.000000,3.000000,4.000000,3.000000,3.000000,2.000000,5.000000,4.000000,4.000000,3.000000,3.000000,4.000000,2.000000
75%,25.000000,4.000000,4.000000,4.000000,5.000000,4.000000,5.000000,4.000000,4.000000,3.000000,6.000000,4.000000,4.000000,4.000000,3.000000,4.000000,3.000000
max,28.000000,5.000000,5.000000,5.000000,7.000000,4.000000,5.000000,4.000000,5.000000,4.000000,6.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000


In [12]:
dat

,age,check_computer,english_first,gender,hand,have_antivirus,have_firewall,malware_risk_1,malware_risk_2,malware_risk_3,...,skill,subject,threat_severity_1,threat_severity_2,threat_severity_3,threat_susceptibility_1,threat_susceptibility_2,threat_susceptibility_3,updated_antivirus,updated_os
0,26,yes,yes,female,right,yes,yes,5,5,5,...,6,Participant_4,5,5,5,5,5,5,yes,yes
1,25,yes,no,male,right,yes,yes,4,4,4,...,6,Participant_5,5,5,5,3,3,3,yes,yes
2,25,no,yes,female,right,no,no,4,2,2,...,5,Participant_6,3,3,2,3,3,2,no,no
3,25,no,no,female,right,no,no,5,5,5,...,5,Participant_7,4,4,4,2,4,2,no,no
4,23,no,yes,female,right,no,no,4,3,4,...,4,Participant_8,4,4,4,3,4,2,no,no
5,20,no,no,male,right,no,no,4,4,4,...,5,Participant_9,4,4,2,2,2,2,no,yes
6,26,yes,yes,female,right,yes,yes,4,2,4,...,6,Participant_10,4,4,3,2,4,4,yes,no
7,20,no,yes,female,right,no,yes,3,4,4,...,5,Participant_11,2,2,2,2,3,2,no,yes
8,21,no,no,female,right,no,no,4,3,4,...,6,Participant_12,4,4,3,3,4,2,no,no
9,18,no,yes,female,right,no,no,4,4,3,...,5,Participant_13,4,3,3,3,4,4,no,no


In [13]:
dat = dat.drop(dat.index[8]) # remove participant 12 as they were removed from analysis

### Demographics

In [14]:
dat['gender'].value_counts()

female    15
male       5
Name: gender, dtype: int64

In [15]:
dat['hand'].value_counts()

right    19
left      1
Name: hand, dtype: int64

In [16]:
dat['english_first'].value_counts()

yes    15
no      5
Name: english_first, dtype: int64

### Characteristics

In [17]:
dat['have_antivirus'].value_counts()

yes    10
no     10
Name: have_antivirus, dtype: int64

In [18]:
dat['updated_antivirus'].value_counts()

no     14
yes     6
Name: updated_antivirus, dtype: int64

In [19]:
dat['updated_os'].value_counts()

no     11
yes     9
Name: updated_os, dtype: int64

In [20]:
dat['check_computer'].value_counts()

no     15
yes     5
Name: check_computer, dtype: int64

In [21]:
dat['have_firewall'].value_counts()

yes    10
no     10
Name: have_firewall, dtype: int64

### Preparing instruments

In [22]:
dat.columns

Index(['age', 'check_computer', 'english_first', 'gender', 'hand',
       'have_antivirus', 'have_firewall', 'malware_risk_1', 'malware_risk_2',
       'malware_risk_3', 'risk_appetite_1', 'risk_appetite_2',
       'risk_appetite_3', 'risk_appetite_4', 'risk_appetite_5',
       'risk_appetite_6', 'skill', 'subject', 'threat_severity_1',
       'threat_severity_2', 'threat_severity_3', 'threat_susceptibility_1',
       'threat_susceptibility_2', 'threat_susceptibility_3',
       'updated_antivirus', 'updated_os'],
      dtype='object')

In [23]:
dat['malware_risk'] = (dat['malware_risk_1'] + dat['malware_risk_2'] + dat['malware_risk_3'])/3
dat['risk_appetite'] = (dat['risk_appetite_1'] + dat['risk_appetite_2'] + dat['risk_appetite_3'] + dat['risk_appetite_4'] + dat['risk_appetite_5'] + dat['risk_appetite_6'])/6
dat['threat_severity'] = (dat['threat_severity_1'] + dat['threat_severity_2'] + dat['threat_severity_3'])/3
dat['threat_susceptibility'] = (dat['threat_susceptibility_1'] + dat['threat_susceptibility_2'] + dat['threat_susceptibility_3'])/3

In [24]:
dat_processed = dat[['subject','check_computer', 'english_first', 'gender', 'hand', 'have_antivirus', 'have_firewall', 'updated_antivirus', 'updated_os', 'malware_risk', 'risk_appetite', 'threat_severity', 'threat_susceptibility']]

In [25]:
dat_processed.head()

,subject,check_computer,english_first,gender,hand,have_antivirus,have_firewall,updated_antivirus,updated_os,malware_risk,risk_appetite,threat_severity,threat_susceptibility
0,Participant_4,yes,yes,female,right,yes,yes,yes,yes,5.000000,4.000000,5.000000,5.000000
1,Participant_5,yes,no,male,right,yes,yes,yes,yes,4.000000,2.500000,5.000000,3.000000
2,Participant_6,no,yes,female,right,no,no,no,no,2.666667,3.166667,2.666667,2.666667
3,Participant_7,no,no,female,right,no,no,no,no,5.000000,2.500000,4.000000,2.666667
4,Participant_8,no,yes,female,right,no,no,no,no,3.666667,3.666667,4.000000,3.000000


### Write to CSV

In [26]:
dat_processed.to_csv('./questionaires.csv')